Разделяем слой с остановками на уникальные маршруты, поскольку есть маршруты в разных областях ЛО c одинаковыми названиями, необходимо применить кластеризацию по расположению с искуственным шумом. Если все маршруты уникальные это не требуется

In [ ]:
from shapely import Point
import pandas as pd
from tqdm.auto import tqdm
import geopandas as gpd
from dongraphio.utils.geometry_utils import clusterize_kmeans_geo_points

tqdm.pandas()

stops_gdf = gpd.read_file("stopsLO.geojson")
stops_gdf.rename(columns={"Автоб": "route"}, inplace=True)
stops_gdf.dropna(subset="route", axis=0, inplace=True, how="any")
unique_routes = set()
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.add(i.lstrip().rstrip())
unique_routes = pd.DataFrame(index=list(unique_routes))
unique_routes["geometry"] = [[]] * len(unique_routes)
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.loc[i.lstrip().rstrip(), "geometry"] = unique_routes.loc[i.lstrip().rstrip()]["geometry"] + [
                row["geometry"]]

devided_routes = pd.DataFrame()
noise_points = [[31.21239, 58.61124], [31.22239, 58.62161], [31.23239, 58.61171], [31.24239, 58.62129],
                [31.23239, 58.64131],
                [32.92133, 59.02899], [32.94933, 59.06599], [32.93943, 59.03599], [32.92733, 59.08999],
                [32.92333, 59.00599],
                [35.58878, 59.48945], [35.58933, 59.48359], [35.58643, 59.489759], [35.58879, 59.48688],
                [35.58869, 59.485749]]
noise_points = [Point(x) for x in noise_points]
for index, row in tqdm(unique_routes.iterrows(), total=unique_routes.shape[0]):
    clusterized = clusterize_kmeans_geo_points(row, noise_points)
    if clusterized.shape[0] > 1:
        clusterized.index = index + "." + clusterized.index.astype(str)
        devided_routes = pd.concat([devided_routes, clusterized])
    else:
        clusterized.index = [index]
        devided_routes = pd.concat([devided_routes, clusterized])

In [ ]:
road_types = {
"['unclassified', 'tertiary']":1.05,
"['trunk', 'trunk_link']":1,
"service":1.35,
"['living_street', 'residential']":1.15,
"['unclassified', 'residential']":1.15,
"['primary_link', 'motorway_link']":1,
"['secondary_link', 'unclassified']":1,
"unclassified":1,
"['tertiary', 'residential']":1.05,
"['service', 'unclassified']":1.3,
"['secondary_link', 'secondary']":1,
"motorway_link":1,
"['service', 'residential']":1.3,
"['service', 'road']":1.25,
"['secondary', 'residential']":1.05,
"rest_area":1.35,
"living_street":1.3,
"motorway":1,
"['service', 'living_street', 'residential']":1.35,
"['service', 'tertiary']":1.3,
"secondary":1,
"trunk":1,
"['service', 'living_street']":1.3,
"tertiary":1,
"['primary', 'primary_link']":1,
"secondary_link":1,
"['service', 'unclassified', 'residential']":1.25,
"trunk_link":1,
"tertiary_link":1,
"['motorway', 'motorway_link']":1,
"['living_street', 'unclassified']":1.15,
"primary_link":1,
"road":1.1,
"primary":1,
"['residential', 'tertiary']":1,
"['residential', 'secondary']":1,
"residential":1.2
}


In [ ]:
import momepy
from shapely import LineString, Polygon
import networkx as nx
import osmnx as ox
from tqdm.auto import tqdm
import geopandas as gpd



tqdm.pandas()
def get_osmnx_graph(graph_type: str, boundary:Polygon ,truncate_by_edge:bool,city_crs:int) -> nx.MultiDiGraph:
        G_ox = ox.graph_from_polygon(
            polygon=boundary, network_type=graph_type, truncate_by_edge=truncate_by_edge
        )
        nodes: gpd.GeoDataFrame
        edges: gpd.GeoDataFrame
        nodes, edges = momepy.nx_to_gdf(G_ox, points=True, lines=True, spatial_weights=False)
        nodes = nodes.to_crs(city_crs).set_index("nodeID")
        nodes_coord = nodes.geometry.apply(
            lambda p: {"x": round(p.coords[0][0], 2), "y": round(p.coords[0][1], 2)}
        ).to_dict()

        edges = edges[["length", "node_start", "node_end", "geometry","highway"]].to_crs(city_crs)
        edges["type"] = graph_type
        edges["geometry"] = edges["geometry"].apply(
            lambda x: LineString([tuple(round(c, 2) for c in n) for n in x.coords] if x else None)
        )
        travel_type = "car"
        speed = 4 * 1000 / 60 if graph_type == "walk" else 17 * 1000 / 60
        G = nx.MultiDiGraph()
        for _, edge in tqdm(edges.iterrows(), total=len(edges), desc=f"Collecting {graph_type} graph", leave=False):
            p1 = int(edge.node_start)
            p2 = int(edge.node_end)
            type = str(edge.highway)
            geom = (
                LineString(
                    (
                        [
                            (nodes_coord[p1]["x"], nodes_coord[p1]["y"]),
                            (nodes_coord[p2]["x"], nodes_coord[p2]["y"]),
                        ]
                    )
                )
                if not edge.geometry
                else edge.geometry
            )
            koef = road_types.get(str(type)) 
            if koef is None:
                koef=1
            G.add_edge(
                p1,
                p2,
                length_meter=edge.length*koef,
                #length_meter=edge.length,
                geometry=str(geom),
            )

        nx.set_node_attributes(G, nodes_coord)
        G.graph["crs"] = "epsg:" + str(city_crs)
        return G
polygon = gpd.read_file('bounds.geojson')['geometry'].item()
drive_graph = get_osmnx_graph('drive_service',polygon,truncate_by_edge=True,city_crs=32636)



In [ ]:

from shapely import from_wkt
import networkx as nx
from src.dongraphio.utils.graph_utils import buffer_directed_roads, nx_to_gdf

for i in drive_graph.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
drive_graph_gdf = nx_to_gdf(drive_graph, nodes=False, edges=True)
drive_graph_gdf_buffer = buffer_directed_roads(drive_graph_gdf, 3)



In [ ]:
devided_routes = gpd.read_file('devided_routes.geojson')
devided_routes['geometry'] = devided_routes['geometry'].apply(lambda x:list(x.geoms))
devided_routes.set_index('index',inplace=True)
devided_routes

In [ ]:
%reload_ext autoreload
%autoreload 2
from src.dongraphio import project_points_on_graph, add_projected_points_as_nodes,resolve_tsp, subgraph_by_path
from src.dongraphio.utils import get_dist_matrix_for_tsp,convert_multidigraph_to_digraph
import networkx as nx

city_crs = 32636
bus_routes = nx.DiGraph()
error_routes = []
for index, row in tqdm(devided_routes.iterrows(), total=devided_routes.shape[0]):
    try:
        geometry_list = row["geometry"]
        row_gdf = gpd.GeoDataFrame(data={"label": [index for _ in geometry_list], "geometry": geometry_list})
        row_gdf = row_gdf.set_crs(city_crs,allow_override=True)

        projected_points = project_points_on_graph(drive_graph_gdf, drive_graph_gdf_buffer, row_gdf)

        route_nodes, graph_with_new_points = add_projected_points_as_nodes(projected_points, drive_graph)
        graph_with_new_points = convert_multidigraph_to_digraph(graph_with_new_points)
        distance_matrix, path_matrix = get_dist_matrix_for_tsp(graph_with_new_points, route_nodes)
        path = resolve_tsp(distance_matrix,5)
        route_nx = subgraph_by_path(path, path_matrix, graph_with_new_points)
        bus_routes = nx.compose(bus_routes, route_nx)

        del graph_with_new_points, projected_points, route_nodes, distance_matrix, path_matrix, path, route_nx
    except Exception as e:
        error_routes.append((index, row,e))
        pass




In [ ]:
print("gg")

In [ ]:
from shapely import MultiPoint

data = [{'route':obj[0],'geometry':MultiPoint(obj[1]['geometry']),'error':str(obj[2])} for obj in error_routes]
errors = gpd.GeoDataFrame(data=data,geometry='geometry',crs=4326)
errors.to_file('Error_routes_2.geojson')


In [ ]:
for i in bus_routes.edges(data=True):
    i[2]['geometry'] = (str(i[2]['geometry']))
# nx.write_graphml(bus_routes, 'bus_LO_all.graphml')
nx.write_gml(bus_routes,'bus_LO_all.gml')

In [ ]:
bus_routes.graph['crs']=32636
for i in bus_routes.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
graph_bus_gdf = nx_to_gdf(nx.MultiDiGraph(bus_routes),edges=True)
graph_bus_gdf.to_file('bus_LO_all.geojson')

In [ ]:
import geopandas as gpd
import pandas as pd
projected_points = gpd.read_file('nodes_.geojson')
projected_points

In [ ]:
grouped = projected_points.groupby('index')['geometry'].apply(list)
grouped = pd.DataFrame(grouped)
grouped

In [ ]:
import numpy as np
real_stops = gpd.read_file('stops_initial.geojson')
real_stops.rename(columns={"Автоб": "route"}, inplace=True)
real_stops.replace('',np.nan,inplace=True)
real_stops.dropna(subset="route", axis=0, inplace=True, how="any")

real_stops

In [ ]:
real_stops.to_crs(32636,inplace=True)


join = gpd.sjoin_nearest(real_stops,projected_points,how='inner',distance_col='dist')
rows_to_del = []
for ind,row in join.iterrows():
    routes = str(row["route"]).replace('.', ',').replace(' ','').replace('5__','5').rstrip().lstrip().upper()
    routes = routes
    routes = routes.split(",")
    if str(row['index']).upper().replace(' ','').split('.')[0] not in routes:
        rows_to_del.append(ind)
join.drop(rows_to_del,inplace=True)
join = pd.DataFrame(data=join.groupby('index')['geometry'].apply(list))
join

In [ ]:
from shapely import MultiPoint
devided_routes = join.copy()
devided_routes['geometry'] = devided_routes['geometry'].apply(lambda x:MultiPoint(x))
devided_routes = gpd.GeoDataFrame(data=devided_routes,crs=32636)
devided_routes.to_file('devided_routes.geojson')

In [ ]:
devided_routes = join.copy()

In [ ]:
from src.dongraphio.base_models import OSMGrapher
graph_osm = OSMGrapher(city_osm_id=1114252,city_crs=32636,keep_city_boundary=False).get_public_trasport_graph()

In [ ]:
import networkx as nx
import geopandas as gpd
from src.dongraphio.utils.graph_utils import nx_to_gdf

graph_osm_gdf = nx_to_gdf(graph_osm,edges=True)
graph_osm_gdf.to_file('graph_LO_OSM_2.geojson')

In [ ]:
graph_osm.graph.clear()

In [ ]:
for _, _, data in graph_osm.edges(data=True):
    data["geometry"] = (str(data["geometry"]))
nx.write_gml(graph_osm,'graph_osm.gml')
